In [2]:
# BTC on-chain (per-block) via Blockstream/Esplora – no API key needed
import requests, time
import pandas as pd
from datetime import datetime, timezone

BASE = "https://blockstream.info/api"   # mainnet
N_BLOCKS = 50                           # how many latest blocks to pull

def get_json(url):
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.json()

def get_tip_height():
    return int(requests.get(f"{BASE}/blocks/tip/height", timeout=15).text)

def get_block_hash(height: int):
    return requests.get(f"{BASE}/block-height/{height}", timeout=15).text.strip()

def get_block_info(block_hash: str) -> dict:
    return get_json(f"{BASE}/block/{block_hash}")

rows = []
tip = get_tip_height()
start_h = max(0, tip - N_BLOCKS + 1)

for h in range(tip, start_h - 1, -1):  # newest → oldest
    try:
        bh = get_block_hash(h)
        b = get_block_info(bh)
        rows.append({
            "height": h,
            "hash": bh,
            "timestamp": datetime.fromtimestamp(b.get("timestamp", 0), tz=timezone.utc),
            "tx_count": b.get("tx_count"),
            "size_bytes": b.get("size"),
            "weight": b.get("weight"),
            "version": b.get("version"),
            "bits": b.get("bits"),
            "merkle_root": b.get("merkle_root"),
            "nonce": b.get("nonce"),
            "prev_block": b.get("prev_hash"),
            "mediantime": datetime.fromtimestamp(b.get("mediantime", 0), tz=timezone.utc) if b.get("mediantime") else None,
        })
        if len(rows) % 50 == 0:
            print(f"Fetched {len(rows)} blocks… latest height {h}")
        time.sleep(0.15)   # be polite
    except Exception as e:
        print(f"[warn] height {h}: {e}")
        time.sleep(0.5)

btc_blocks = pd.DataFrame(rows).sort_values("height").reset_index(drop=True)
btc_blocks.to_csv("btc_blocks_latest.csv", index=False)
try:
    btc_blocks.to_parquet("btc_blocks_latest.parquet", index=False)
except Exception as e:
    print(f"[parquet skip] {e}")

print(btc_blocks.tail(3))

Fetched 50 blocks… latest height 910537
    height                                               hash  \
47  910584  0000000000000000000192a49544b21352a1902bfaac49...   
48  910585  000000000000000000019ee6252952eb5e8c8c45cd61c3...   
49  910586  00000000000000000001d13b30bc72483fb95dc62b4413...   

                   timestamp  tx_count  size_bytes   weight    version  \
47 2025-08-18 09:42:05+00:00      3751     2363812  3993691  537280512   
48 2025-08-18 10:06:25+00:00      3484     1588561  3993589  611246080   
49 2025-08-18 10:26:37+00:00      3549     1586232  3993915  547356672   

         bits                                        merkle_root       nonce  \
47  386018483  be633a29400e78c8fb61d725abdf36d7afc8cb72b705b0...  2901414244   
48  386018483  20e19847bcebac0b0d892b8e0b4788e3967800615ea686...  1357426306   
49  386018483  b09cc4dbf663579f899a01af5cd0a5f8067b736456ad55...  3852746858   

   prev_block                mediantime  
47       None 2025-08-18 09:12:16+00:00

In [3]:
daily = (btc_blocks
         .assign(date=btc_blocks["timestamp"].dt.floor("D"))
         .groupby("date", as_index=False)
         .agg(tx_count_day=("tx_count","sum"),
              blocks=("height","count"),
              avg_weight=("weight","mean"),
              avg_size=("size_bytes","mean")))
daily.to_csv("btc_daily_block_stats.csv", index=False)